## ENG1585 - Projeto Final - Fundo de Índice

### Grupo:
 - Gabriel Alcouffe (2320011)
 - Luisa Compasso (2110976)
 - Matheus Nogueira (1810764)


## Importação dos pacotes

In [ ]:
using JuMP, CSV, DataFrames, Statistics, Plots, Cbc

## Importação dos dados

In [ ]:
retornos = CSV.read("retornos_projeto.csv", DataFrame);
precos = CSV.read("precos_projeto.csv", DataFrame);

datas = retornos[:,1];

y = retornos.CAC40
R = retornos[:,3:end];

## Primeira Versão - DR

In [ ]:
function downside_risk_optimization(y::Vector{Fl}, R::Matrix{Fl}; j::Int64=1) where{Fl}
    T, N = size(R)
    model = JuMP.Model(Ipopt.Optimizer)
    set_silent(model)
    @variable(model, x[1:N] >= 0)
    @variable(model, θ[1:T]>= 0) 
    @constraint(model, [t in 1:T], θ[t]>= y[t] - x'R[t,:])
    @constraint(model, sum(x)==1)
    @objective(model, Min, sum((θ[t])^2 for t in T-j:T))
    optimize!(model)

    return value.(x), value.(θ), objective_value(model)
end

## Segunda Versão - DR com custos de transação

In [ ]:
function downside_risk_optimization_with_costs(y::Vector{Fl}, R::Matrix{Fl}, W::Float64, alocacaoFinanceira::Vector{Fl};
                                                j::Int64=1, C::Float64=1.) where{Fl}
    T, N = size(R)
    model = JuMP.Model(Ipopt.Optimizer)
    set_silent(model)
    @variable(model, x[1:N] >= 0)
    @variable(model, θ[1:T] >= 0) 
    @variable(model, c[1:N] >= 0)
    @constraint(model, [t in 1:T], θ[t]>= y[t] - x'R[t,:])
    @constraint(model, sum(x)==1)

    @constraint(model, [i in 1:N], c[i] >= x[i]*W.-alocacaoFinanceira[i])
    @constraint(model, [i in 1:N], c[i] >= -(x[i]*W.-alocacaoFinanceira[i]))
    @constraint(model, 0.01*sum(c) <= C*W)
    
    @objective(model, Min, sum((θ[t])^2 for t in j:T))
    optimize!(model)

    x_opt = value.(x)
    # turnover_opt = sum(abs.(x_opt*W.-alocacaoFinanceira))

    return x_opt, value.(θ), objective_value(model)#, turnover_opt
end

## Terceira Versão - DR com custos de transação e esparsidade

In [ ]:
function downside_risk_sparse_optimization_with_costs(y::Vector{Fl}, R::Matrix{Fl}, W::Float64, alocacaoFinanceira::Vector{Fl};
                                                j::Int64=1, k::Int64=38, λ::Float64=0.,
                                                initial_I::Vector{Int64}=zeros(38)) where{Fl}
    T, N = size(R)
    model = JuMP.Model(SCIP.Optimizer)
    set_silent(model)
    @variable(model, x[1:N] >= 0)
    @variable(model, θ[T-j:T] >= 0) 
    @variable(model, c[1:N] >= 0)
    @variable(model, I[1:N], Bin)
    set_start_value.(I, initial_I)
    @constraint(model, [t in T-j:T], θ[t]>= y[t] - x'R[t,:])
    @constraint(model, sum(x)==1)
    # Sparse
    @constraint(model, [i in 1:N], x[i] <= I[i])
    @constraint(model, sum(I)<=k)

    @constraint(model, [i in 1:N], c[i] >= x[i]*W.-alocacaoFinanceira[i])
    @constraint(model, [i in 1:N], c[i] >= -(x[i]*W.-alocacaoFinanceira[i]))
    # @constraint(model, 0.01*sum(c) <= C*W)
    
    @objective(model, Min, sum((θ[t]) for t in T-j:T) + λ*sum(c))
    optimize!(model)

    x_opt = value.(x)

    return x_opt, value.(θ), value.(I), objective_value(model)
end

## Funções auxiliares de plots

In [ ]:
function plot_comparacao_retorno(datas_backtest,retornos_carteira, y, H, j, k, λ)
    plot(datas_backtest,retornos_carteira,label="Carteira")
    plot!(datas_backtest,y[end-H+1:end], label="Índice")
    plot!(xlabel="Dias", ylabel="", title="Comparação retornos diários (j = $j, k = $k, λ = $λ)")
end

function plot_turnover(turnover, j, k, λ)
    p1 = plot(100*turnover,seriestype=:bar,label="")
    p1 = plot!(ylabel="Turnover (%)", xlabel="Dias", title="Turnover Diário Percentual")
    
    p2 = plot(turnover.*W[2:end],seriestype=:bar,label="")
    p2 = plot!(ylabel="Turnover (\$)", xlabel="Dias", title="Turnover Diário Monetário")
    
    plot(p1, p2,  layout=grid(2,1), size=(900,800), plot_title = "", title=["Turnover Diário Percentual (j = $j, k = $k, λ = $λ)" "Turnover Diário Monetário (j = $j, k = $k, λ = $λ)"])

end

function plot_W(datas_backtest, y, W, j, k, λ)
    
    W_indice = [10000.]
    for h in 1:H
        push!(W_indice, W_indice[end]+y[end-H+h]*W_indice[end])
    end
    
    plot(title="Comparação Evolução Monetária (j = $j, k = $k, λ = $λ)")
    plot!(datas_backtest, W[2:end], label="Carteira")
    plot!(datas_backtest, W_indice[2:end], label="Índice")
end

## Otimização e backtest

In [ ]:
janelas          = [7, 30, 90, 200]
λs               = [0.,  0.5, 1e-4, 1e-6]
k_values         = [10, 20, 30, 38]
H                = 50
N                = size(R, 2)
datas_backtest   = datas[end-H+1:end]
W                = [10000.]
alocacao_inicial = ones(N).*W[1]/N

combinations = collect(Iterators.product(janelas, λs, k_values))

alocacaoFinanceira_opt = []
retornos_carteira_opt  = []
alocacao_opt           = []
erros_opt              = []
turnover_opt           = []
θ_values_opt           = []
W_opt                  = []
j_opt                  = 0
λ_opt                  = 0.
k_opt                  = 0

min_error = Inf

df_hyperparameters = DataFrame(Dict("j"=>-1,"k"=>-1,"λ"=>-1.,"error"=>-Inf))


In [ ]:
for (j, λ, k) in combinations
    println("Otimizando j=$j, λ=$(λ), k=$k")
    W = [10000.]
    alocacaoFinanceira = [alocacao_inicial]
    T = length(y) - H
    retornos_carteira = []
    alocacao = []
    erros = []
    turnover = []
    θ_values = []
    I_values = [Int64.(zeros(N))]

    for h in 1:H
        println("Janela $h")
        y_T = y[1:T]
        R_T = Matrix(R[1:T,:])
        R_futuro = Vector(R[T+1,:])
        y_futuro = y[T+1]

        x_opt, θ_opt, I_opt, obj_value = downside_risk_sparse_optimization_with_costs(y_T, R_T, W[end], alocacaoFinanceira[end];j=j, k=k, λ=λ, initial_I=I_values[end])

        W_futuro_bruto = W[end]*(1+ x_opt'R_futuro)
        turnovert = abs(W_futuro_bruto - W[end])
        custo_transacao = 0.01*turnovert
        W_liquido = W_futuro_bruto - custo_transacao

        push!(W, W_liquido)
        push!(θ_values, θ_opt)
        push!(I_values, round.(I_opt))

        T += 1

        alocFincanceiraT = x_opt*W[end]
        push!(alocacaoFinanceira, alocFincanceiraT)
        push!(turnover, turnovert/W[end-1])    
        push!(alocacao, x_opt)
        # push!(erros, obj_value)
        push!(erros, (y_futuro - x_opt'R_futuro)^2)
        push!(retornos_carteira, x_opt'R_futuro)
    end

    push!(df_hyperparameters, Dict(Dict("j"=>j,"k"=>k,"λ"=>λ,"error"=>mean(erros))))
    CSV.write("hyperparameters.csv", df_hyperparameters)
    
    if mean(erros) <= min_error
        min_error = mean(erros)
        j_opt = j
        k_opt = k
        λ_opt = λ
        error_opt = erros
        retornos_carteira_opt = retornos_carteira
        alocacao_opt = alocacao
        turnover_opt = turnover
        W_opt = W
        alocaco_financeira_opt = alocacaoFinanceira
    end
    
    plot_comparacao_retorno(datas_backtest,retornos_carteira, y, H, j, k, λ)
    savefig("modelo_final_comparacao_retornos_$(j)$(k)$(λ).png")

    plot_turnover(turnover, j, k, λ)
    savefig("modelo_final_turnovers_$(j)$(k)$(λ).png")

    plot_W(datas_backtest, y, W, j, k, λ)
    savefig("modelo_final_comparacao_W_$(j)$(k)$(λ).png")
end